## CMPINF 2110 Spring 2022 - Week 03

### Joining/Merging data 
 
Joining two or more data sets together is a critical step in many data analysis projects. Joins represent that different data objects are **related** in some way. Our question of interest requires that we bring information from the various data sets together. This notebook introduces joins from a data analysis context. The examples mostly come from Chapter 13 of the [R for Data Science](https://r4ds.had.co.nz/relational-data.html) (R4DS) by Hadley Wickham. I highly recommend that you read that chapter. I have converted the introductory examples from R to Python using Pandas. The discussion and pictures within the R4DS book clearly depict the mechanisms of data joins. I will sometimes make use of the phrases and text within R4DS in the discussion of the notebook and so please take this statement as an unofficial citation of the excellent R4DS book.

As we will see in the weeks to come, joins are an essential aspect of **relational data**. We will discuss the **relational data model** in more detail later on, however many of the concepts can be demonstrated with DataFrames before we start working with MySQL.

## Import modules

The four primary modules that we work with are imported below.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

## Introduction to joins

The following examples come from section 13.4.1 from [R4DS](https://r4ds.had.co.nz/relational-data.html#understanding-joins).

Let's create 2 simple DataFrames, `x` and `y`. Both contain 3 rows and 2 columns. They each have a `key` column which uniquely identifies an observation (a row). An observation contained in the `x` DataFrame records the value of the `val_x` variable, while an observation in the `y` DataFrame records the value of the `val_y` variable.

In [9]:
x = pd.DataFrame({'key': [1, 2, 3], 'val_x': ['x1', 'x2', 'x3']})

x

,key,val_x
0,1,x1
1,2,x2
2,3,x3


In [3]:
y = pd.DataFrame({'key': [1, 2, 4], 'val_y': ['y1', 'y2', 'y3']})

y

,key,val_y
0,1,y1
1,2,y2
2,4,y3


We wish to combine the `x` and `y` data sets together. We can think of a few ways to do so, such as concatenating the two vertically using the `pd.concat()` function.

In [4]:
pd.concat([x, y], axis=0)

,id,val_x,key,val_y
0,1.0,x1,NaN,NaN
1,2.0,x2,NaN,NaN
2,3.0,x3,NaN,NaN
0,NaN,NaN,1.0,y1
1,NaN,NaN,2.0,y2
2,NaN,NaN,4.0,y3


Or concatenating horizontally by specifying `axis=1`.

In [5]:
pd.concat([x, y], axis=1)

,id,val_x,key,val_y
0,1,x1,1,y1
1,2,x2,2,y2
2,3,x3,4,y3


Although vertical and horizontal concatenations are appropriate in some contexts (we have used vertical concat several times this semester), neither seem to yield satisfactory results for this example. We wish to **join** or **merge** the two data sets to allow us to examine the behavior of `val_x` **and** `val_y` for the **same** observation. Do either of the above results allow us to do that?

The appropriate way to **join** the data sets together is to match the rows based on the `key` value. The Pandas User Guide calls this style of joining [Database-style DataFrame joining/merging](https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging) (I recommend reading over that portion of the Pandas User Guide for a Pandas focused introduction to joins). The "Database-style" name refers to the fact that these joins are inspired by and make use of SQL concepts and ideas. We will discuss SQL in the weeks to come.  

The "database-style" joins can be performed one of two ways. The `pd.merge()` function can be used to join or merge a "left" DataFrame with a "right" DataFrame. The "left" and "right" terms are associated with the argument order in the function call:  

`pd.merge( <LeftDataFrame>, <RightDataFrame> )`

The second approach is to access the `.merge()` method associated with a DataFrame object. In this setup the "left" DataFrame is the object itself.

### Inner-join

Let's start with the default joining method in `pd.merge()`, the **inner-join**. An inner-join matches the rows based on the value of the key variable. Thus, in our simple example we want to combine the `val_x` and `val_y` columns whenever `key` is the same in the two DataFrames. Before executing the `pd.merge()` function, let's look at our DataFrames again. Which rows share the `key` value between the two data sets?

In [10]:
x

,key,val_x
0,1,x1
1,2,x2
2,3,x3


In [7]:
y

,key,val_y
0,1,y1
1,2,y2
2,4,y3


An inner-join uses the **intersection** of the shared `key` values between the two DataFrames. Thus, the only rows remaining after the join operation are those that have the same `key` values appearing in **both** DataFrames. Which rows do you expect to see in the **joined** DataFrame between `x` and `y`?

Let's see what happens. The cell below executes the `pd.merge()` function and explicitly defines the `how` argument as `'inner'`, even though it is the default choice. The key variable (column) is specified by the `on` argument within the `pd.merge()` function.

In [11]:
pd.merge( x, y, on='key', how='inner')

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2


As we can see above, there are only **two** remaining rows. The `key = 3` value in `x` does **not** appear in `y` and so it was removed. Likewise, the `key = 4` value in `y` does **not** appear in `x` and so was removed. 

Let's now see this again, but this time using the `.merge()` method on the `x` object. Again, the key variable (column) is specified by the `on` argument. The output is displayed to the screen and shows we have the same joined data set that we had with the `pd.merge()` function call.

In [12]:
x.merge( y, on='key', how='inner')

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2


### Left-join

A left-join performs the join and keeps all rows in the **left** data set. Rows from the right data set are joined **only** if the `key` value is shared in the left data set. Because of this, if a `key` in the left data set **does not** appear in the right data set, the joined data set **contains missings**.

Left-joins are my preferred way of joining two data sets, especially when I am just starting out with a data analysis project. I like to think of a left-join as merging a smaller DataFrame (the right data set) to a larger DataFrame (the left data set). Thus, I will typically refer to left-joins as "merging a smaller object to a larger one".

Let's see how the left-join works with our simple example. The left-join keeps all rows in the left data set. We will treat `x` as the left object and so the remaining joined data set will have `key` values of `1`, `2`, and `3`. Thus, the `key = 4` row in the `y` data set will **not** be present in the final data set.

In [13]:
pd.merge( x, y, on='key', how='left')

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN


The output DataFrame consists of 3 rows. We have all 3 observations from `x`, as we expected. Notice that the last row has a **missing** value for `val_y`. That is because `key = 3` does **not** exist in `y`. However, because we used a left-join the final DataFrame **must** contain a row for `key = 3`. Since no such observation exists in the right data set for that case, a missing value is recorded for the variables brought in from the right data set.

### Right-join

Right-joins operate just like left-joins, except the final joined data set keeps all rows from the right data set. Let's perform this operation for our simple example.

In [14]:
pd.merge( x, y, on='key', how='right')

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2
2,4,NaN,y3


We have 3 rows in our merged DataFrame because `y` has 3 rows. Notice the `key` values are `1`, `2`, and `4` because those are `key` values contained in `y`. With a right-join, the `val_x` column now contains a missing because `key = 4` does **not** appear in the `x` DataFrame.

### Full-join

A full-join keeps all observations from **both** DataFrames. We should thus expect to see missing values whenever the key values in one data set do not appear in the other. The final merged data set will have as many rows as total unique values for the key variable(s) in both data sets. 

**How many rows do you expect a full-join data set will have for our simple example?**

As a reminder, let's look at the `key` column from the two data sets.

In [15]:
x.key

0    1
1    2
2    3
Name: key, dtype: int64

In [16]:
y.key

0    1
1    2
2    4
Name: key, dtype: int64

How many unique values are there?

A full join is specified in the `pd.merge()` function by setting the `how` argument to `'outer'`. Let's see what happens.

In [17]:
pd.merge( x, y, on='key', how='outer' )

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN
3,4,NaN,y3


As we can see above, the result consists of 4 rows. The `val_y` column contains a missing value for the `key = 3` row for the same reason we had a missing value in `val_y` in the left-join case. Likewise, the `val_x` column has a missing value for the `key = 4` row for the same reason we had a missing value in `val_x` in the right-join case. A full-outer join highlights that the merger is based on the **key variable(s)**. Simple vertical and horizontal concats could not replicate this combined DataFrame.

The previous joins - inner, left, and right - are subsets of the full-join. The inner-join keeps rows that appear in both DataFrames. The left and right joins are themselves both outer joins because at least one DataFrame has all the rows kept. The full-join is the full-outer-join because all rows in both DataFrames are retained.

### Key variables

The `pd.merge()` function specifies the **key variable(s)** with the `on` argument. With our simple example, we have a single key variable named `key`. That was why the previous code examples set `on='key'` in the `pd.merge()` function calls. Let's get some additional practice understanding the behavior of the key variable by walking through the [Duplicate keys section in the R4DS book](https://r4ds.had.co.nz/relational-data.html#join-matches) in Section 13.4.4.

The cell below defines two new DataFrames, `xb` and `yb`. Both consist of a `key` column which defines each observation. The `xb` DataFrame contains the `val_x` variable and the `yb` DataFrame contains the `val_y` variable. Notice that the `xb` DataFrame has repeated or duplicated `key` values, while the `key` is unique in `yb`.

In [18]:
xb = pd.DataFrame({'key': [1, 2, 2, 1], 'val_x': ['x1', 'x2', 'x3', 'x4']})

xb

,key,val_x
0,1,x1
1,2,x2
2,2,x3
3,1,x4


In [19]:
yb = pd.DataFrame({'key': [1, 2], 'val_y': ['y1', 'y2']})

yb

,key,val_y
0,1,y1
1,2,y2


What will happen if we join these two data sets? The `key` value is not unique to `xb`. **Will we get an error?**

Let's execute a left-join and see what happens. We will join "on" the `key` column and so the `on` argument is set to `'key'` in the `pd.merge()` function call in the cell below.

In [20]:
pd.merge( xb, yb, on='key', how='left')

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2
2,2,x3,y2
3,1,x4,y1


The join worked! To help understand what's going on, let's repeat the left-join and rearrange the columns of the final data set.

In [21]:
pd.merge( xb, yb, on='key', how='left').loc[:, ['val_x', 'key', 'val_y']]

,val_x,key,val_y
0,x1,1,y1
1,x2,2,y2
2,x3,2,y2
3,x4,1,y1


With the `key` column as the middle of the three columns in the joined data set, we can more easily see that `key` is defining one observation in the `y` data set. Our left-join was therefore aligning the `val_x` variable with the `val_y` variable based on `key`, even though `key` does **not** uniquely define one observation in `xb`. The `key` variable is therefore known as a **primary key** for the `yb` DataFrame, because it uniquely defines the row in `yb`. However, the `key` variable is a **foreign key** in the `xb` DataFrame because it uniquely defines an observation in a different DataFrame.  

Primary and foreign keys are important **relational data model** concepts that we will return to time and time again. 

The cell below defines creates two additional DataFrames, `xc` and `yc`. Both DataFrames have a `key` column, but notice the `key` values are duplicated in both DataFrames! The `key` variable therefore does not uniquely identify the rows in either DataFrame. The R4DS book states that this may represent an error. Alternatively, we can view the `key` variable as a primary key in some other DataFrame that is not available.

In [22]:
xc = pd.DataFrame({'key': [1, 2, 2, 3], 'val_x': ['x1', 'x2', 'x3', 'x4']})

xc

,key,val_x
0,1,x1
1,2,x2
2,2,x3
3,3,x4


In [23]:
yc = pd.DataFrame({'key': [1, 2, 2, 3], 'val_y': ['y1', 'y2', 'y3', 'y4']})

yc

,key,val_y
0,1,y1
1,2,y2
2,2,y3
3,3,y4


Joining the `xc` and `yc` DataFrames `on` the `'key'` column produces all possible combinations of the rows in the two data sets. We have more rows than unique values for the `key` column because neither data set has the rows uniquely identified. The join procedure does not really "know" what to do, and so all possible ways the observations could be joined are provided.

In [24]:
pd.merge( xc, yc, on='key', how='left')

,key,val_x,val_y
0,1,x1,y1
1,2,x2,y2
2,2,x2,y3
3,2,x3,y2
4,2,x3,y3
5,3,x4,y4


### Context

If you were in CMPINF 2100, you may remember that I mentioned I begin any Exploratory Data Analysis (EDA) by counting. I count the number of rows associated with different variables (columns) and combinations of variables. One of the reasons I do that is to identify and/or confirm the variables that **define** an observation. Understanding the **primary key(s)** is critical when we go about merging data sets, as you saw in these series of simple examples. We may be surprised when we join data sets when we do not know what uniquely defines a row. The last example with `xc` and `yc` revealed a joined data set with more rows than either originally contained. Although managable with this small example, imagine a moderate sized application where one data set has 1000 rows and another has 5000 rows. It may be difficult to understand what happened after the fact when we get surprised by the sizes of a joining operation.  

For those reasons, it is critical to first check the number of rows associated with the key variable(s). With `xc` and `yc` we can easily confirm that there are more than 1 row associated with the `key = 2` value.

In [25]:
xc.key.value_counts()

2    2
1    1
3    1
Name: key, dtype: int64

In [26]:
yc.key.value_counts()

2    2
1    1
3    1
Name: key, dtype: int64

If we check the number of rows associated with each value of `key` in the joined data set, we see the impact of the duplicated key values.

In [27]:
pd.merge( xc, yc, on='key', how='left' ).key.value_counts()

2    4
1    1
3    1
Name: key, dtype: int64

## Realistic example

Let's practice joins using a subset of the nyc-flights data set discussed in R4DS. The cell below reads in three CSV files. The first is a data set consisting of flights from NYC during the summer 2013 (R4DS uses the complete version of this data set consisting of all months in 2013). The second is information about specific planes, and the third is a data set related to airlines. These three files are available for download on Canvas. Please download them and save them within the same local working directory as your Jupyter notebook to run the examples.

In [29]:
flights2 = pd.read_csv( "summer_flights.csv" )

planes = pd.read_csv( "planes.csv" )

airlines = pd.read_csv( "airlines.csv" )

### `flights2` data

Let's look at the `.info()` and the first few rows of the summer flights data set.

In [30]:
flights2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86995 entries, 0 to 86994
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   year     86995 non-null  int64 
 1   month    86995 non-null  int64 
 2   day      86995 non-null  int64 
 3   hour     86995 non-null  int64 
 4   origin   86995 non-null  object
 5   dest     86995 non-null  object
 6   tailnum  86267 non-null  object
 7   carrier  86995 non-null  object
dtypes: int64(4), object(4)
memory usage: 5.3+ MB


In [31]:
flights2.head(10)

,year,month,day,hour,origin,dest,tailnum,carrier
0,2013,6,1,23,JFK,PSE,N618JB,B6
1,2013,6,1,5,EWR,CLT,N538UW,US
2,2013,6,1,5,EWR,IAH,N35407,UA
3,2013,6,1,5,LGA,IAH,N27724,UA
4,2013,6,1,5,JFK,BQN,N806JB,B6
5,2013,6,1,5,JFK,MIA,N5EAAA,AA
6,2013,6,1,6,EWR,RSW,N492UA,UA
7,2013,6,1,6,LGA,DFW,N3EUAA,AA
8,2013,6,1,6,LGA,PHL,N946UW,US
9,2013,6,1,6,JFK,IAD,N835AS,EV


As discussed in R4DS, we may be tempted to think that each `tailnum` has one unique flight per date (combination of `year`, `month`, `day`). However, that is not true! The cell below shows a simple way to identify the number of rows associated with each group using a combination of `.groupby()`, `.size()`, and `.value_counts()`. As we see below, there are 48665 unique date and `tailnum` combinations. However, there are 12274 date and `tailnum` combinations with two rows in `flights2`. Interestingly, there are 37 date and `tailnum` combinations with 5 rows!

In [32]:
flights2.groupby(['year', 'month', 'day', 'tailnum']).size().\
reset_index(name='num_rows').\
num_rows.value_counts()

1    48665
2    12274
3     3083
4      905
5       37
Name: num_rows, dtype: int64

Let's see a few of the date and `tailnum` combinations are associated with 5 rows.

In [33]:
flights2.groupby(['year', 'month', 'day', 'tailnum']).size().\
reset_index(name='num_rows').\
sort_values(['num_rows'], ascending=False).head()

,year,month,day,tailnum,num_rows
29546,2013,7,13,N368JB,5
40884,2013,7,29,N329JB,5
37362,2013,7,24,N346JB,5
11951,2013,6,18,N16961,5
25255,2013,7,7,N374JB,5


As a final check, the cell below selects the rows associated with one date and `tailnum` combination that appears 5 times in `flights2`.

In [34]:
flights2.loc[ (flights2.month == 7) & (flights2.day == 13) & (flights2.tailnum == "N368JB") ]

,year,month,day,hour,origin,dest,tailnum,carrier
39526,2013,7,13,6,JFK,BUF,N368JB,B6
39708,2013,7,13,10,JFK,BOS,N368JB,B6
39872,2013,7,13,13,JFK,MVY,N368JB,B6
40055,2013,7,13,17,JFK,PWM,N368JB,B6
40224,2013,7,13,21,JFK,MSY,N368JB,B6


As we can see above, the `hour` variable is different for all 5 rows. It appears that particular plane has multiple flights each day throughout New England and the East Coast. So maybe each observation in the `flights2` is uniquely defined as the `year`, `month`, `day`, and `hour` associated with a particular `tailnum`. Let's check.

In [35]:
flights2.groupby(['year', 'month', 'day', 'hour', 'tailnum']).size().\
reset_index(name='num_rows').\
num_rows.value_counts()

1    86060
2      102
3        1
Name: num_rows, dtype: int64

We're close, but there are still 102 `year`, `month`, `day`, `hour`, and `tailnum` combinations that appear twice in `flights2`. There is even 1 particular combination of the variables that appears 3 times in `flights2`. Let's see which combination of the variables that corresponds to.

In [36]:
flights2.groupby(['year', 'month', 'day', 'hour', 'tailnum']).size().\
reset_index(name='num_rows').\
sort_values(['num_rows'], ascending=False).head()

,year,month,day,hour,tailnum,num_rows
37038,2013,7,11,6,N607JB,3
33343,2013,7,7,9,N723TW,2
30392,2013,7,3,15,N724MQ,2
37074,2013,7,11,7,N318JB,2
25340,2013,6,28,6,N877AS,2


The cell below selects the rows associated with the `year`, `month`, `day`, `hour` and `tailnum` combination with 3 rows in the original data set. As we can see below, those values do appear in 3 separate rows (as illustrated by the different `.index` values). **Is there a variable (column) that changes across the rows displayed below?**

In [37]:
flights2.loc[ (flights2.month == 7) & (flights2.day == 11) & (flights2.hour == 6) & (flights2.tailnum == "N607JB") ]

,year,month,day,hour,origin,dest,tailnum,carrier
37515,2013,7,11,6,JFK,TPA,N607JB,B6
38431,2013,7,11,6,JFK,MCO,N607JB,B6
38432,2013,7,11,6,JFK,FLL,N607JB,B6


The `dest` column has three separate values! So it seems like we need to include `dest` within our collection of variables that uniquely define a row. Let's check this is indeed the case.

In [38]:
flights2.groupby(['year', 'month', 'day', 'hour', 'origin', 'dest', 'tailnum']).size().\
reset_index(name='num_rows').\
num_rows.value_counts()

1    86267
Name: num_rows, dtype: int64

As we can see above, we have identified what one row represents. One row, or observation, is a `tailnum` between an `origin` and `dest` on a given `year`, `month`, `day`, and `hour`. The **primary key** is thus a **composite** of 7 variables!

### `planes` data

Let's now identify one row in the `planes` DataFrame. We will start by looking at the `.info()` and `.head()`.

In [39]:
planes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   tailnum       3322 non-null   object 
 1   year          3252 non-null   float64
 2   type          3322 non-null   object 
 3   manufacturer  3322 non-null   object 
 4   model         3322 non-null   object 
 5   engines       3322 non-null   int64  
 6   seats         3322 non-null   int64  
 7   speed         23 non-null     float64
 8   engine        3322 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 233.7+ KB


In [40]:
planes.head(10)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan
5,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
6,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
7,N108UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
8,N109UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
9,N110UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan


Let's check if the `tailnum` uniquely defines one observation.

In [41]:
planes.groupby(['tailnum']).size().reset_index(name='num_rows').num_rows.value_counts()

1    3322
Name: num_rows, dtype: int64

Yes! One row corresponds to a value of `tailnum`. Thus, `tailnum` is the **primary key** of `planes`. This also means that `tailnum` is a **foreign key** within `flights2`. It allows us to *link* or **relate** to information contained in another data set.

### `airlines` data

Let's now look at the `airlines` data to see if we can identify what one observation corresponds to. The `.info()` method reveals that `airlines` is quite small with just 16 rows.

In [42]:
airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   carrier  16 non-null     object
 1   name     16 non-null     object
dtypes: object(2)
memory usage: 384.0+ bytes


Let's look at the entire data set. As we can see below, the `carrier` variable is associated with a full or longer name for each airline. 

In [43]:
airlines

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.
5,EV,ExpressJet Airlines Inc.
6,F9,Frontier Airlines Inc.
7,FL,AirTran Airways Corporation
8,HA,Hawaiian Airlines Inc.
9,MQ,Envoy Air


We can confirm that one row corresponds to one unique value of `carrier`.

In [45]:
airlines.groupby(['carrier']).size().reset_index(name='num_rows').num_rows.value_counts()

1    16
Name: num_rows, dtype: int64

The `carrier` variable is the **primary key** in `airlines` and is a **foreign key** in `flights2`. We can use the `airlines` data set to "decode" what the abbrevation or nickname is for each airline within `flights2`.

### Flights per manufacturer

Let's now use joins to answer a specific question. We would like to know the number of flights associated with each manufacturer. The `flights2` data set gives us information associated with each flight. The manufacturer of the plane is not stored in that data set. We must join `flights2` and `planes` on `tailnum` in order to answer this question. Since we are only interested in the manufacturer, let's create a smaller version of `planes` consisting of just two columns.

In [46]:
plane_makers = planes.loc[:, ['tailnum', 'manufacturer']].copy()
plane_makers

,tailnum,manufacturer
0,N10156,EMBRAER
1,N102UW,AIRBUS INDUSTRIE
2,N103US,AIRBUS INDUSTRIE
3,N104UW,AIRBUS INDUSTRIE
4,N10575,EMBRAER
...,...,...
3317,N997AT,BOEING
3318,N997DL,MCDONNELL DOUGLAS AIRCRAFT CO
3319,N998AT,BOEING
3320,N998DL,MCDONNELL DOUGLAS CORPORATION


Let's now merge `plane_makers` with `flights2` on `tailnum` using a left-join.

In [47]:
flights_b = pd.merge( flights2, plane_makers, on='tailnum', how='left')

In [50]:
flights_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86995 entries, 0 to 86994
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          86995 non-null  int64 
 1   month         86995 non-null  int64 
 2   day           86995 non-null  int64 
 3   hour          86995 non-null  int64 
 4   origin        86995 non-null  object
 5   dest          86995 non-null  object
 6   tailnum       86267 non-null  object
 7   carrier       86995 non-null  object
 8   manufacturer  73463 non-null  object
dtypes: int64(4), object(5)
memory usage: 6.6+ MB


The total number of flights associated with each manufacturer can be found by counting the number of rows per unique manufacturer. As we can see below, over 22,000 flights are associated with Boeing, followed by Embraer, and then Airbus.

In [51]:
flights_b.manufacturer.value_counts()

BOEING                           22239
EMBRAER                          17308
AIRBUS                           12107
AIRBUS INDUSTRIE                  9625
BOMBARDIER INC                    6958
MCDONNELL DOUGLAS AIRCRAFT CO     2516
MCDONNELL DOUGLAS                 1146
CANADAIR                           416
MCDONNELL DOUGLAS CORPORATION      384
GULFSTREAM AEROSPACE               190
CESSNA                             127
ROBINSON HELICOPTER CO              64
BARKER JACK L                       64
PIPER                               57
CIRRUS DESIGN CORP                  50
FRIEDEMANN JON                      26
DEHAVILLAND                         24
CANADAIR LTD                        21
BELL                                19
STEWART MACO                        18
KILDALL GARY                        18
LAMBERT RICHARD                     16
MARZ BARRY                          11
LEBLANC GLENN T                     11
AMERICAN AIRCRAFT INC                9
AGUSTA SPA               

### Manufacturers per airline

What if we would like to know how many unique manufacturers each airline uses? We need to use group by the `carrier` variable and then apply the `nunique()` function.

In [52]:
flights_b.groupby(['carrier']).\
aggregate(num_makers = ('manufacturer', 'nunique')).\
reset_index()

,carrier,num_makers
0,9E,1
1,AA,22
2,AS,1
3,B6,6
4,DL,6
5,EV,3
6,F9,2
7,FL,2
8,HA,1
9,MQ,3


Although we have tecnically answered our question, I do not know what each `carrier` abbrevation stands for. We can merge the `airline` data set on `carrier` to bring in the full or long `name` associated with each airline. The cell below uses the `.merge()` method associated with a DataFrame to demonstrate **chaining together multiple steps**.

In [53]:
flights_b.groupby(['carrier']).\
aggregate(num_makers = ('manufacturer', 'nunique')).\
reset_index().\
merge(airlines, on='carrier', how='left')

,carrier,num_makers,name
0,9E,1,Endeavor Air Inc.
1,AA,22,American Airlines Inc.
2,AS,1,Alaska Airlines Inc.
3,B6,6,JetBlue Airways
4,DL,6,Delta Air Lines Inc.
5,EV,3,ExpressJet Airlines Inc.
6,F9,2,Frontier Airlines Inc.
7,FL,2,AirTran Airways Corporation
8,HA,1,Hawaiian Airlines Inc.
9,MQ,3,Envoy Air


We are now able to see that American Airlines uses 22 unique manufacturers, while Jetblue uses just 6.

## Summary

Joins or mergers between data sets allow us to combine information stored within different data sets. Those data sets are *related* to each other via certain variables that allow the joining or linking to take place. We have thus started to think about answering data analysis questions not based on a single large data set, but through *relationships* defined across multiple data sets. As we will see in the weeks to come, storing data in such a way streamlines and simplifies the management and preservation of data over time. We have thus started thinking about creating a **data base** to efficiently store data for long term use!